In [6]:
using DataFrames
using CSV

In [8]:
function mean(x)
    sum(x)/length(x)
end

mean (generic function with 1 method)

# Load MPSGE Solutions

In [38]:
file_name = "FullReportCounter"

replace_dict = Dict(
    "Y" => [r"^Y\[\(:(\w{3}),\)]", s"\1"],
    "A" => [r"^A\[\(:(\w{3}),\)]", s"\1"],
    "MS" => [r"^MS\[\(:(\w{3}),\)]", s"\1"],
    "PA" => [r"^PA\[\(:(\w{3}),\)]", s"\1"],
    "PY" => [r"^PY\[\(:(\w{3}),\)]", s"\1"],
    "PVA" => [r"^PVA\[\(:(\w{6,7}),\)]", s"\1"],
    "PM" => [r"^PM\[\(:(\w{3}),\)]", s"\1"],
    "PFX" => [r"^PFX", s""],
    "RA" => [r"^RA", s""],
    );

d = DataFrame(CSV.File("./Results/"*file_name*".csv",stringtype=String)) |>
    x -> transform(x, :margin => (y -> ifelse.(y.==".","0",y)) => :margin) |>
    x -> transform(x, :margin => (y -> parse.(Float64,y))=>:margin);


df_mpsge = DataFrame()

for (var,(reg,sub)) in replace_dict
    df_ = d |>
        x-> filter(:var_name => y -> occursin.(reg ,y),x) |>
        x -> transform(x,
            :var_name => (y -> replace.(y, reg => sub)) => :domain,
            :var_name => (y -> var) => :var
        ) |>
        x -> select(x, [:var_name,:var,:domain,:value,:margin])
    
    df_mpsge = vcat(df_mpsge,df_)
end

df_mpsge = df_mpsge |>
    x -> select(x, [:var, :domain, :value]);

# Create comparison dataframe

In [39]:
df_mpsge

Row,var,domain,value
,String,String,Float64
1,Y,ppd,1.0188
2,Y,res,1.03916
3,Y,com,0.99922
4,Y,amb,0.969242
5,Y,fbp,1.04402
6,Y,rec,1.02558
7,Y,con,0.998728
8,Y,agr,1.02651
9,Y,eec,0.993423


In [27]:
df_mcp = DataFrame(CSV.File("./Results/national_sols_julia_mcp.csv",stringtype=String))
    
df = DataFrame(CSV.File("./Results/national_sols_gams.csv",stringtype=String)) |>
    x -> dropmissing(x) |>
    x -> outerjoin(df_mpsge,df_mcp,x, on = [:var,:domain],makeunique=true) |>
    x -> rename(x, :value => :julia_mpsge, :value_1 => :julia_mcp, :value_2 => :gams) |>
    x -> transform(x, [:julia_mpsge,:julia_mcp] => ((a,b) -> abs.(a-b)) => :diff) |>
    #x -> filter(:julia_mcp => y -> !ismissing(y) && y !=0 , x) |>
    x -> sort(x,:diff;rev=true)

Row,var,domain,julia_mpsge,julia_mcp,gams,diff
,String,String,Float64?,Float64?,Float64?,Float64?
1,PA,fbt,missing,0.0,1.0,missing
2,PA,mvt,missing,0.0,1.0,missing
3,PA,gmt,missing,0.0,1.0,missing
4,RA,,12453.9,missing,missing,missing
5,PFX,,0.97386,missing,missing,missing
6,PFX,‡A[ppd],25.2364,missing,missing,missing
7,PFX,‡A[res],2.73798,missing,missing,missing
8,PFX,‡A[com],30.0287,missing,missing,missing
9,PFX,‡A[amb],0.0863405,missing,missing,missing


In [35]:
df[80:120,:]

Row,var,domain,julia_mpsge,julia_mcp,gams,diff
,String,String,Float64?,Float64?,Float64?,Float64?
1,PFX,†A→dmins,52.4084,missing,missing,missing
2,PFX,†A→dmtex,31.6662,missing,missing,missing
3,PFX,†A→dmleg,3.90076,missing,missing,missing
4,PFX,†A→dmfen,0.291135,missing,missing,missing
5,PFX,†A→dmuti,2.38156,missing,missing,missing
6,PFX,†A→dmnmp,25.9611,missing,missing,missing
7,PFX,†A→dmbrd,0.15539,missing,missing,missing
8,PFX,†A→dmbnk,0.0453854,missing,missing,missing
9,PFX,†A→dmedu,1.75023,missing,missing,missing


# Full Julia MPSGE Solution

In [21]:
replace_dict = Dict(
    "Y" => [r"^Y\[\(:(\w{3}),\)]", s"(:\1,)"],
    "A" => [r"^A\[\(:(\w{3}),\)]", s"(:\1,)"],
    "MS" => [r"^MS\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PA" => [r"^PA\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PY" => [r"^PY\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PVA" => [r"^PVA\[\(:(\w{6,7}),\)]", s"(:\1,)"],
    "PM" => [r"^PM\[\(:(\w{3}),\)]", s"(:\1,)"],
    "PFX" => [r"^PFX", s""],
    "RA" => [r"^RA", s""],
    "C" => [r"^PA\[(\w{3})\]ρRA", s"(:\1,)"],
    "S_PY_Y" => [r"^PY\[(\w{3})\]‡Y\[(\w{3})\]", s"(:\1,:\2)"],
    "D_PA_Y" => [r"^PA\[(\w{3})\]†Y\[(\w{3})\]", s"(:\1,:\2)"],
    "D_PVA_Y" => [r"^PVA\[(\w{3})\]†Y→VA(\w{3})", s"(:\1,:\2)"],
    "S_PM_MS" => [r"^PM\[(\w{3})\]‡MS\[(\w{3})\]", s"(:\1,:\2)"],
    "D_PY_MS" => [r"^PY\[(\w{3})\]†MS\[(\w{3})\]", s"(:\1,:\2)"],
    "S_PA_A" => [r"^PA\[(\w{3})\]‡A\[(\w{3})\]", s"(:\1,:\2)"],
    "S_PFX_A" => [r"^PFX‡A\[(\w{3})\]", s"(:\1,)"],
    "D_PY_A" => [r"^PY\[(\w{3})\]†A→dm(\w{3})", s"(:\1,:\2)"],
    "D_PFX_A" => [r"^PFX†A→dm(\w{3})", s"(:\1,)"],
    "D_PM_A" => [r"^PM\[(\w{3})\]†A\[(\w{3})\]", s"(:\1,:\2)"],
    );

In [23]:
d = DataFrame(CSV.File("./Results/FullReportCounter.csv",stringtype=String)) |>
    x -> transform(x, :margin => (y -> ifelse.(y.==".","0",y)) => :margin) |>
    x -> transform(x, :margin => (y -> parse.(Float64,y))=>:margin);

In [24]:

df = DataFrame()

for (var,(reg,sub)) in replace_dict
    df_ = d |>
        x-> filter(:var_name => y -> occursin.(reg ,y),x) |>
        x -> transform(x,
            :var_name => (y -> replace.(y, reg => sub)) => :domain,
            :var_name => (y -> var) => :var
        ) |>
        x -> select(x, [:var_name,:var,:domain,:value,:margin])
    
    df = vcat(df,df_)
end

In [25]:
df

Row,var_name,var,domain,value,margin
,String,String,String,Float64,Float64
1,PA[ppd]†Y[ppd],D_PA_Y,"(:ppd,:ppd)",43.8969,-0.0
2,PA[res]†Y[ppd],D_PA_Y,"(:res,:ppd)",0.162683,-0.0
3,PA[com]†Y[ppd],D_PA_Y,"(:com,:ppd)",0.197119,-0.0
4,PA[fbp]†Y[ppd],D_PA_Y,"(:fbp,:ppd)",1.66646,-0.0
5,PA[rec]†Y[ppd],D_PA_Y,"(:rec,:ppd)",0.00511639,-0.0
6,PA[con]†Y[ppd],D_PA_Y,"(:con,:ppd)",0.668667,-0.0
7,PA[eec]†Y[ppd],D_PA_Y,"(:eec,:ppd)",0.454946,-0.0
8,PA[pub]†Y[ppd],D_PA_Y,"(:pub,:ppd)",0.00411802,-0.0
9,PA[ins]†Y[ppd],D_PA_Y,"(:ins,:ppd)",0.234656,-0.0
